In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

theta = 'pi/4'


In [ ]:
folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

Purify_XX.get_tstamps_and_offsets(contains = 'XX')
Purify_YY.get_tstamps_and_offsets(contains = 'YY')
Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')


In [ ]:
### load the data
Purify_XX.load_raw_data()
Purify_XX.correct_pq_times()
Purify_YY.load_raw_data()
Purify_YY.correct_pq_times()
Purify_ZZ.load_raw_data()
Purify_ZZ.correct_pq_times()

In [ ]:
def sweep_parameter(param_name,sweep_pts,apply_ROC, plot_all = False):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
      
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    psi_F = 0.5*(psi_m_F+psi_p_F)
    psi_F_u = 0.5*np.sqrt(psi_m_F_u**2+psi_p_F_u**2)
    
#     print 'Overall fidelities ', psi_F[:3]
#     print psi_F_u[:3]
    
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity')
    plt.errorbar(sweep_pts,psi_m_F,psi_m_F_u,label = '-')
    plt.errorbar(sweep_pts,psi_p_F,psi_p_F_u,label = '+')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend()
    plt.show()
    plt.close('all')
 
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity (avg)')
    plt.errorbar(sweep_pts,psi_F,psi_F_u)
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.show()
    plt.close('all')
    
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('ebits')

    plt.errorbar(sweep_pts,ebits_p,ebits_p_u,label = '+')
    plt.errorbar(sweep_pts,ebits_m,ebits_m_u,label = '-')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend()
    plt.show()
    plt.close('all')
    
    ### get the rate for any given data point
    ### keep in mind that this includes ALL down time of the experiment. Some might argue that this is an unfair comparison
    ### we therefore also include a diagram that uses the operation time of the sequence only and the number of times the sequence has run
    ### tomography is excluded from this calculated rate
    total_time = 0
    total_time += Purify_XX.get_total_time() + Purify_YY.get_total_time() + Purify_ZZ.get_total_time()
    total_counts = psi_m_XX[0]+psi_p_XX[0]+psi_m_YY[0]+psi_p_YY[0]+psi_m_ZZ[0]+psi_p_ZZ[0]
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Rate (Hz)')
    plt.plot(sweep_pts,total_counts/total_time, label = 'incl. overhead')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50]) 
    total_time = 0
    total_time += Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    plt.plot(sweep_pts,total_counts/total_time, label = 'excl. overhead')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend(loc = 2)
    plt.show()
    plt.close('all')
    
    ## plot individual correlations
    if plot_all:
        fig  = plt.figure()
        ax = plt.subplot()
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_m_XX[1],psi_m_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_m_YY[1],psi_m_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_m_ZZ[1],psi_m_ZZ[2],label = 'ZZ')
        if param_name == 'bin_w2':
            plt.xlim(np.array(plt.xlim()) + [-50,50])   
        plt.title('Psi_minus correlations')
        plt.legend()
        plt.show()
        plt.close('all')
        
        fig  = plt.figure()
        ax = plt.subplot()
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_p_XX[1],psi_p_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_p_YY[1],psi_p_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_p_ZZ[1],psi_p_ZZ[2],label = 'ZZ')
        if param_name == 'bin_w2':
            plt.xlim(np.array(plt.xlim()) + [-50,50])
        plt.title('Psi_plus correlations')
        plt.legend()
        plt.show()
        plt.close('all')

In [ ]:
sweep_parameter('bin_w2',np.arange(50,450,50),apply_ROC = True,plot_all = True)

In [ ]:
sweep_parameter('st_start',np.linspace(-4e3,4e3,10)+2773.5e3,apply_ROC = True)

In [ ]:
sweep_parameter('max_reps_w1',np.arange(100,1000,50),apply_ROC = True)

In [ ]:
sweep_parameter('min_cr_lt4_after',np.arange(1,20,2),apply_ROC = True)
sweep_parameter('min_cr_lt3_after',np.arange(1,20,2),apply_ROC = True)

In [ ]:
sweep_parameter('max_dt',np.linspace(2.5e3,40e3,10),apply_ROC = True)

In [ ]:
sweep_parameter('st_len',np.linspace(3e3,40e3,10),apply_ROC = True)